In [1]:
from tqdm import tqdm
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import os
import sys

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.abspath(''))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils import set_api_key

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'distilbert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(model_name)

In [7]:
pinecone_key  = set_api_key('PINECONE_API_KEY') 
pc = Pinecone(api_key=pinecone_key, ssl_verify=False)

API key found in .env file for PINECONE_API_KEY


In [8]:
pc.list_indexes()

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[]

In [11]:
from pinecone import ServerlessSpec

pc.create_index(
    name = 'vector-demo',
    metric='euclidean',
    dimension=768,
    spec= ServerlessSpec(cloud='aws', region='us-east-1')
)

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
    "name": "vector-demo",
    "metric": "euclidean",
    "host": "vector-demo-wosks6d.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}

In [12]:
data = [
    {"id": "vector1",  "text": "I love using vector databases"},
    {"id": "vector2",  "text": "Vector databases are great for storing and retrieving vectors"},
    {"id": "vector3",  "text": "Using vector databases makes my life easier"},
    {"id": "vector4",  "text": "Vector databases are efficient for storing vectors"},
    {"id": "vector5",  "text": "I enjoy working with vector databases"},
    {"id": "vector6",  "text": "Vector databases are useful for many applications"},
    {"id": "vector7",  "text": "I find vector databases very helpful"},
    {"id": "vector8",  "text": "Vector databases can handle large amounts of data"},
    {"id": "vector9",  "text": "I think vector databases are the future of data storage"},
    {"id": "vector10", "text": "Using vector databases has improved my workflow"}
]

This code snippet iterates over each sentence in the data list we defined earlier and encodes the text of each sentence into a vector using the downloaded sentence transformer model (model).

In [13]:
vector_data = []

for sentence in data:
    embedding = model.encode(sentence['text'])
    vector_info = {'id': sentence['id'], 'values':embedding.tolist()}
    vector_data.append(vector_info)

In [20]:
index = pc.Index('vector-demo')

In [21]:
index.upsert(vectors=vector_data)

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vector-demo-wosks6d.svc.aped-4627-b74a.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'upserted_count': 10}

In [22]:
index.describe_index_stats()

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vector-demo-wosks6d.svc.aped-4627-b74a.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10,
 'vector_type': 'dense'}

Now that we have stored the vectors in the above index, let’s run a similarity search to see the obtained results.

We can do this using the query() method of the index object we created earlier.

First, we define a search text and generate its embedding:

In [27]:
search_text = 'I don"t know what I would do without vector databases'
search_embeddings = model.encode(search_text).tolist()

In [28]:
index.query(vector=search_embeddings, top_k=3)

d:\AI Projects\AI agent using LangGraph\AI-agent-using-langGraph\ai-agent-langgraph-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vector-demo-wosks6d.svc.aped-4627-b74a.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'matches': [{'id': 'vector3', 'score': 177.036774, 'values': []},
             {'id': 'vector9', 'score': 189.69101, 'values': []},
             {'id': 'vector6', 'score': 199.439606, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}